In [2]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

In [4]:
inter_df = pd.read_csv("../../data/1-preparation/interaction/interaction_chembl.csv")
inter_df = inter_df.drop(columns=['SMILES', 'FASTA Sequence'])
inter_df

,UniProt ID,Drug
0,Q9UBK2,COLFORSIN
1,Q9UBK2,DEXAMETHASONE
2,O15379,VORINOSTAT
3,O15379,TACEDINALINE
4,O15379,DACINOSTAT
...,...,...
674,O14894,"4,4'-DIHYDROXYCHALCONE"
675,P81277,KISSPEPTIN-10
676,P81277,DAMGO
677,P81277,NALOXONE


In [6]:
maccs = pd.read_csv('../../data/2-feature/mol/maccs_nrlmf_chembl.csv')
maccs

,Drug,MACCSFP1,MACCSFP2,MACCSFP3,MACCSFP4,MACCSFP5,MACCSFP6,MACCSFP7,MACCSFP8,MACCSFP9,...,MACCSFP157,MACCSFP158,MACCSFP159,MACCSFP160,MACCSFP161,MACCSFP162,MACCSFP163,MACCSFP164,MACCSFP165,MACCSFP166
0,COLFORSIN,0,0,0,0,0,0,0,0,0,...,1,0,1,1,0,0,1,1,1,0
1,DEXAMETHASONE,0,0,0,0,0,0,0,0,0,...,1,0,1,1,0,0,1,1,1,0
2,VORINOSTAT,0,0,0,0,0,0,0,0,0,...,0,1,1,0,1,1,1,1,1,0
3,TACEDINALINE,0,0,0,0,0,0,0,0,0,...,0,1,1,1,1,1,1,1,1,0
4,DACINOSTAT,0,0,0,0,0,0,0,0,0,...,1,1,1,0,1,1,1,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
456,"4,4'-DIHYDROXYCHALCONE",0,0,0,0,0,0,0,0,0,...,1,0,1,0,0,1,1,1,1,0
457,KISSPEPTIN-10,0,0,0,0,0,0,0,0,0,...,1,1,1,1,1,1,1,1,1,0
458,DAMGO,0,0,0,0,0,0,0,0,0,...,1,1,1,1,1,1,1,1,1,0
459,NALOXONE,0,0,0,0,0,0,0,0,0,...,1,1,1,0,1,1,1,1,1,0


In [7]:
aaindex = pd.read_csv('../../data/2-feature/prot/aaindex_nrlmf_chembl.csv')
aaindex

,UniProt ID,ANDN920101,ARGP820101,ARGP820102,ARGP820103,BEGF750101,BEGF750102,BEGF750103,BHAR880101,BIGC670101,...,KARS160113,KARS160114,KARS160115,KARS160116,KARS160117,KARS160118,KARS160119,KARS160120,KARS160121,KARS160122
0,Q9UBK2,3.066707,-2.170256,-1.977858,-2.333132,-2.488069,-2.677095,2.738514,2.638519,-0.334663,...,0.786476,1.497195,1.333951,1.710537,1.116361,1.790829,1.253216,0.093426,1.453492,1.391010
1,O15379,1.383463,0.391879,-0.959167,-0.895293,-0.823573,0.666852,0.722144,-0.242320,0.448794,...,0.904216,0.622215,0.815818,0.684566,1.251251,-0.314495,0.867716,-0.476666,0.726892,-0.836357
2,P36894,0.160383,0.446220,0.006955,-0.031379,-0.234900,0.376213,0.147584,-0.092977,0.645348,...,0.406457,0.735826,0.615034,0.866117,0.649316,0.935664,0.686485,-0.791727,0.532325,-0.506287
3,P20393,1.550796,-1.720927,-0.168155,-0.463940,-1.956911,-1.388424,1.839802,0.917593,-1.603182,...,-0.793959,-0.580371,-0.518858,-0.462731,-0.854349,0.762912,-0.338028,0.458311,0.074486,2.004152
4,P06213,0.671017,0.138592,-0.442222,-0.534896,-0.752958,-0.382287,0.846182,0.265544,0.340824,...,0.528128,0.582875,0.541818,0.653083,0.695783,0.337997,0.739079,0.002996,0.735284,0.368676
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
138,Q5NUL3,-1.668818,2.598595,2.728783,2.778199,1.956135,2.724477,-2.644934,-2.349186,1.556375,...,0.558923,-0.385446,-0.429205,-0.762463,-0.030775,-0.103047,0.091890,-0.122846,0.044905,0.615119
139,Q9H093,-0.091986,-0.431134,-0.850523,-0.889586,-0.457664,-1.314357,0.877325,1.213361,-0.410766,...,-0.008074,0.091502,0.122063,0.260857,-0.240885,-0.867749,-0.750275,0.933385,-0.442723,0.228327
140,Q96P68,0.449759,2.457485,2.295890,2.158853,0.374100,2.402465,-1.457472,-2.448792,1.659945,...,0.783185,0.266564,0.627526,0.147723,0.799098,1.555960,1.348129,-2.907567,1.299479,0.729973
141,O14894,-1.646841,0.029187,2.121777,1.724249,-0.375711,0.849929,0.514484,-1.778334,-1.817757,...,-2.871880,-2.333196,-2.698247,-2.041874,-2.252202,-0.356339,-1.938327,0.833921,-2.549134,-0.326071


In [18]:
# Pivot the dataframe to the desired format
chemogen = inter_df.pivot_table(index='UniProt ID', columns='Drug', aggfunc='size', fill_value=0)
chemogen

Drug,(+)-EHNA,"(E)-3,4,3',5'-TETRAMETHOXYSTILBENE",(R)-9s,(R)-PIA,(R)-THIORPHAN,(S)-PIA,(S)-THIORPHAN,"(Z)-3,4,3',5'-TETRAMETHOXYSTILBENE","1,3-DIPROPYL-8-CYCLOPENTYLXANTHINE [DPCPX]","2,4-DIHYDROXYBENZOPHENONE",...,XERUBORBACTAM,XL-019,ZAPRINAST,ZD-4190,ZD-6169,ZIDOVUDINE,ZILEUTON,ZSTK-474,p-NITROPHENYLPHOSPHATE,succinyl-CoA
UniProt ID,,,,,,,,,,,,,,,,,,,,,
O14686,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
O14894,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
O15264,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
O15379,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
O15519,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Q9UMX1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Q9Y233,1,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
Q9Y478,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [19]:
chemogen.to_csv("../../data/3-nrlmf/1-input_nrlmf/matriks-chemogenomic_chembl.csv", index=True)

In [10]:
chemogen.sum().sum()

679

In [11]:
prot_id = aaindex.drop_duplicates(subset=['UniProt ID']).iloc[:,:1]
prot_id = prot_id['UniProt ID']
prot_id

0      Q9UBK2
1      O15379
2      P36894
3      P20393
4      P06213
        ...  
138    Q5NUL3
139    Q9H093
140    Q96P68
141    O14894
142    P81277
Name: UniProt ID, Length: 143, dtype: object

In [12]:
# Calculate cosine similarity for unique proteins
protein_similarity_matrix = cosine_similarity(aaindex.iloc[:, 1:])
protein_similarity_df = pd.DataFrame(protein_similarity_matrix, index=prot_id, columns=prot_id)
protein_similarity_df

UniProt ID,Q9UBK2,O15379,P36894,P20393,P06213,P35354,P31040,P08235,Q9NTG7,P13945,...,O75751,Q01469,P41134,Q12770,P10997,Q5NUL3,Q9H093,Q96P68,O14894,P81277
UniProt ID,,,,,,,,,,,,,,,,,,,,,
Q9UBK2,1.000000,0.124033,-0.304538,0.703053,0.528564,-0.082183,0.055235,0.658694,-0.170334,-0.651657,...,-0.901269,0.191961,-0.138962,-0.357468,-0.499048,-0.868908,0.765716,-0.721297,-0.608353,-0.101723
O15379,0.124033,1.000000,0.472547,0.037663,0.748333,0.695818,0.094985,0.122090,-0.103689,-0.342581,...,-0.016447,-0.109970,-0.604647,-0.031207,-0.375754,-0.122732,-0.204805,0.199457,0.062177,-0.246098
P36894,-0.304538,0.472547,1.000000,-0.490480,0.326475,0.501958,-0.107800,-0.336675,-0.491087,-0.175350,...,0.335484,0.295886,-0.479159,-0.245005,0.196147,0.450608,-0.667792,0.659123,0.330075,-0.190226
P20393,0.703053,0.037663,-0.490480,1.000000,0.350182,-0.161858,0.256173,0.943384,0.267330,-0.112879,...,-0.523553,-0.184052,0.163461,0.225290,-0.339245,-0.699213,0.620324,-0.533549,-0.085914,0.129407
P06213,0.528564,0.748333,0.326475,0.350182,1.000000,0.538327,-0.014559,0.393773,-0.141708,-0.466266,...,-0.330743,0.027838,-0.570713,-0.104932,-0.403117,-0.389679,0.140253,-0.066264,-0.103027,-0.137088
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Q5NUL3,-0.868908,-0.122732,0.450608,-0.699213,-0.389679,0.212889,-0.329353,-0.665979,0.042426,0.636961,...,0.902511,-0.220592,-0.076655,0.315661,0.566723,1.000000,-0.848141,0.896835,0.477721,0.076651
Q9H093,0.765716,-0.204805,-0.667792,0.620324,0.140253,-0.342823,0.196563,0.523463,0.191997,-0.416286,...,-0.786808,-0.010630,0.175053,-0.159909,-0.438217,-0.848141,1.000000,-0.900847,-0.552793,0.217907
Q96P68,-0.721297,0.199457,0.659123,-0.533549,-0.066264,0.473888,-0.360450,-0.446538,-0.144136,0.471122,...,0.814569,-0.224738,-0.313071,0.278548,0.520949,0.896835,-0.900847,1.000000,0.502521,-0.108054


In [14]:
protein_similarity_df.to_csv("../../data/3-nrlmf/1-input_nrlmf/prot-simmilarity_chembl.csv", index=True)

In [15]:
mol_id = maccs.drop_duplicates(subset=['Drug']).iloc[:,:1]
mol_id = mol_id['Drug']
mol_id

0                   COLFORSIN
1               DEXAMETHASONE
2                  VORINOSTAT
3                TACEDINALINE
4                  DACINOSTAT
                ...          
456    4,4'-DIHYDROXYCHALCONE
457             KISSPEPTIN-10
458                     DAMGO
459                  NALOXONE
460                NOCICEPTIN
Name: Drug, Length: 461, dtype: object

In [16]:
# Calculate cosine similarity for unique proteins
molecule_similarity_matrix = cosine_similarity(maccs.iloc[:, 1:])
molecule_similarity_df = pd.DataFrame(molecule_similarity_matrix, index=mol_id, columns=mol_id)
molecule_similarity_df

Drug,COLFORSIN,DEXAMETHASONE,VORINOSTAT,TACEDINALINE,DACINOSTAT,SPLITOMICIN,ROMIDEPSIN,TRICHOSTATIN,APICIDIN,UBENIMEX,...,PHENOXYBENZAMINE,SCHISANTHERIN A,SCHISANTHEROL A,TETRACYCLINE,LINOLENIC ACID,"4,4'-DIHYDROXYCHALCONE",KISSPEPTIN-10,DAMGO,NALOXONE,NOCICEPTIN
Drug,,,,,,,,,,,,,,,,,,,,,
COLFORSIN,1.000000,0.776396,0.415128,0.322832,0.427948,0.604165,0.585769,0.400028,0.583266,0.561644,...,0.401130,0.714488,0.720930,0.652111,0.545595,0.539164,0.567454,0.544630,0.643325,0.570937
DEXAMETHASONE,0.776396,1.000000,0.452911,0.299382,0.510252,0.442326,0.543220,0.392792,0.574705,0.498202,...,0.488240,0.640503,0.646997,0.642540,0.569210,0.500000,0.596399,0.602197,0.615840,0.602495
VORINOSTAT,0.415128,0.452911,1.000000,0.585517,0.770329,0.333890,0.551447,0.716541,0.631586,0.641026,...,0.368549,0.275086,0.293032,0.492155,0.501280,0.377426,0.556121,0.615868,0.522976,0.599501
TACEDINALINE,0.322832,0.299382,0.585517,1.000000,0.462910,0.401286,0.484322,0.593914,0.529050,0.616334,...,0.348025,0.360668,0.381529,0.617213,0.215166,0.453609,0.549021,0.608006,0.340459,0.571440
DACINOSTAT,0.427948,0.510252,0.770329,0.462910,1.000000,0.417959,0.584094,0.639209,0.750680,0.556349,...,0.571187,0.417392,0.427948,0.535714,0.478091,0.440959,0.696143,0.715868,0.727393,0.638311
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"4,4'-DIHYDROXYCHALCONE",0.539164,0.500000,0.377426,0.453609,0.440959,0.688062,0.374634,0.436436,0.450749,0.452911,...,0.464991,0.588968,0.575108,0.535450,0.368932,1.000000,0.497000,0.550395,0.545275,0.426006
KISSPEPTIN-10,0.567454,0.596399,0.556121,0.549021,0.696143,0.491398,0.722867,0.440196,0.726424,0.734874,...,0.550562,0.542387,0.567454,0.629844,0.416025,0.497000,1.000000,0.800762,0.658281,0.896718
DAMGO,0.544630,0.602197,0.615868,0.608006,0.715868,0.486908,0.654979,0.551075,0.788051,0.703849,...,0.700040,0.514851,0.544630,0.715868,0.491436,0.550395,0.800762,1.000000,0.672927,0.797993


In [17]:
molecule_similarity_df.to_csv("../../data/3-nrlmf/1-input_nrlmf/mol-simmilarity_chembl.csv", index=True)